###  Using API to run the simulation

We use a strongly defined HTTP API protocol to communicate with a C12 simulator (get and receive the data). This tutorial will briefly present basic API calls that can be used for communication with the C12 simulator. On top of our APIs, different backends can be made to be consistent with other quantum computer frameworks and interfaces.

To run each API, we need to pass the unique bearer token that will be used to verify the user's right to access C12's system.

For communication with C12's system, it is possible to use the API library that encapsulates the HTTP communication with the C12 system using the python request module.

To start using it, we need to create an instance of the Request class, as shown in the following code snippet.

In [1]:


from c12simulator_clients.api.client import Request

# UserConfig class
from c12simulator_clients.user_configs import UserConfigs
import os
user_auth_token = os.getenv("C12_TOKEN")
# Create the request instance
# Constructor of the Request class also accepts the verbose parameter, which can be use for more detailed output of the methods.

request = Request(auth_token=user_auth_token, verbose=False)

#### Run the simulation

The method `start_job()` is used to start the simulation. It has the following arguments:
- `qasm_str`: QASM string with quantum circuit
- `shots`:  Number of trials for the simulation
- `result`: what is desired output (statevector, counts, density_matrix)
- `backend_name`: the name of the backend to run on

In [4]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)


qasm_str = circuit.qasm()
job_uuid = request.start_job(
    qasm_str=qasm_str, shots=100, result="density_matrix", backend_name="c12sim"
)

# The function returns the UUID and QASM of started job or HTTP error in case of problems
print(f"job uuid: {job_uuid[0]}")

job uuid: aab2cde9-7a8d-4e27-9e2d-3d46f92382bc


#### Get the results of a simulation

To get the simulation results, we can use the `get_job_result()` method that accepts one mandatory argument (job UUID string) and two optional ones.
- `timeout`, which represents the number of seconds that the function will wait for job completion
- `wait`, number of seconds between each check if the job is finished

If we do not specify a timeout argument the function will run until the result is obtained

In [5]:
job_result = request.get_job_result(job_uuid[0], timeout=120, wait=5)
print(job_result)

{'status': 'FINISHED', 'results': {'density_matrix': [['(0.49949282437085923-5.493376355137836e-21j)', '(-0.00020111820936445123+0.00015178560957347653j)', '(-8.597498004723175e-05-3.188946233230461e-05j)', '(0.4991873191038138+0.00017373392510153137j)'], ['(-0.00020111820936445123-0.00015178560957342102j)', '(0.00040860052509848754-2.5661881505485718e-21j)', '(3.234086099218603e-05-4.7868039037765916e-05j)', '(0.00010915149728408598-3.189802772829431e-05j)'], ['(-8.597498004723175e-05+3.188946233229073e-05j)', '(3.2340860992130516e-05+4.7868039037765895e-05j)', '(0.0002993751253901472+2.1041523155011936e-21j)', '(-0.00022496660571114015+6.986924686594453e-05j)'], ['(0.4991873191038138-0.00017373392510153137j)', '(0.00010915149728408597+3.189802772829431e-05j)', '(-0.0002249666057111402-6.986924686591678e-05j)', '(0.49979919765436065-4.208983176866398e-21j)']]}, 'errors': ''}


#### Get the status of the job

In [6]:
status = request.get_job_status(job_uuid=job_uuid[0])
print(status)

finished


#### Get all jobs

Method `get_user_jobs()` can be used for obtaining information about all the jobs that have been run on the system for the user whose token is passed to the Request instance.

The method has two arguments:
- `limit`: an integer that represents the number of results to obtain
- `offset`: an integer that represents the offset from the first result

Using these two values, one can obtain the pagination, where the offset represents a page and limits the number of results per page. Method returns an array with results.


In [7]:
number_of_results = 5  # Limit parameter, number of records to retrieve
offset = 0
count = 1
while True:
    jobs = request.get_user_jobs(limit=number_of_results, offset=offset)

    for job in jobs:
        print(f"{count:}")
        print(job)
        count += 1

    if len(jobs) < number_of_results:
        break

    offset += number_of_results

1
{'uuid': '5fb57102-ca9e-45dd-875c-cc11a55ca073', 'time': '2023-03-30T13:51:53.976613', 'status': 'finished', 'options': {'shots': 10000, 'result': 'counts,statevector'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(pi/2) q[0];\nrx(-1.8157749899217606) q[1];\nrz(-pi) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(pi) q[0];\nrz(-pi) q[1];\nrx(-1.3258176636680312) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n', 'errors': '0', 'result': '2'}
2
{'uuid': '796158fa-d61b-4dee-8d58-8b1d51897ae5', 'time': '2023-03-30T13:52:43.875645', 'status': 'finished', 'options': {'shots': 10000, 'result': 'counts,statevector'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(-pi/2) q[0];\nrz(-pi) q[0];\nrx(1.815774989921761) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(-pi/2) q[0];\nrz(-2.3713483251754366) q[1];\nry(0.3447011797986471) q[1];\nrz(-2.3713483251754415) q[1];\ncrx(pi) q[0],q[1];\nrz(-pi/2) q[0];\nry(-pi/2) q[0

#### Get information about a specific job

Using the method `get_job()` it is possible to retrieve the information about the specific job with a given UUID.

In [8]:
job_data = request.get_job(job_uuid[0])
print(job_data)

{'uuid': 'aab2cde9-7a8d-4e27-9e2d-3d46f92382bc', 'time': '2023-05-25T08:14:08.244385', 'status': 'finished', 'options': {'shots': 100, 'result': 'density_matrix'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(pi/2) q[0];\nrx(-1.8157749899217606) q[1];\nrz(-pi) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(pi) q[0];\nrz(-pi) q[1];\nrx(-1.3258176636680312) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n', 'errors': '', 'result': {'statevector': ['(0.49949282437085923-5.493376355137836e-21j)', '(0.00040860052509848754-2.5661881505485718e-21j)', '(0.0002993751253901472+2.1041523155011936e-21j)', '(0.49979919765436065-4.208983176866398e-21j)'], 'density_matrix': [['(0.49949282437085923-5.493376355137836e-21j)', '(-0.00020111820936445123+0.00015178560957347653j)', '(-8.597498004723175e-05-3.188946233230461e-05j)', '(0.4991873191038138+0.00017373392510153137j)'], ['(-0.00020111820936445123-0.00015178560957342102j)', '(0.00040